# 音频分类

音频分类与文本分类类似，都是从输入数据中分配一个类别标签。不同之处在于，音频分类处理的是原始音频波形，而不是文本。音频分类的一些实际应用包括识别说话人的意图、语言分类，甚至是通过声音识别动物种类。

本指南将向你展示如何：

1. 在 [MInDS-14](https://huggingface.co/datasets/PolyAI/minds14) 数据集上微调 [Wav2Vec2](https://huggingface.co/facebook/wav2vec2-base) 模型，以分类说话人的意图。
2. 使用微调后的模型进行推理。

要查看与此任务兼容的所有架构和检查点，建议查阅 [任务页面](https://huggingface.co/tasks/audio-classification)。

在开始之前，请确保你已经安装了所有必要的库：


In [ ]:
pip install transformers datasets evaluate


我们鼓励你登录你的 Hugging Face 账户，以便你可以上传并分享你的模型。当提示你时，输入你的令牌来登录：


In [ ]:
from huggingface_hub import notebook_login

notebook_login()


## 加载 MInDS-14 数据集

首先从 🤗 Datasets 库加载 MInDS-14 数据集：


In [ ]:
from datasets import load_dataset, Audio

minds = load_dataset("PolyAI/minds14", name="en-US", split="train")


使用 [train_test_split](https://huggingface.co/docs/datasets/v3.0.2/en/package_reference/main_classes#datasets.Dataset.train_test_split) 方法将数据集的 `train` 分割成较小的训练集和测试集。这可以让你在实验和确保一切正常工作之前，不必花费太多时间在完整数据集上。


In [ ]:
minds = minds.train_test_split(test_size=0.2)


然后查看数据集：


In [ ]:
minds


输出：


虽然数据集中包含了很多有用的信息，如 `lang_id` 和 `english_transcription`，但在本指南中，我们将重点关注 `audio` 和 `intent_class`。使用 [remove_columns](https://huggingface.co/docs/datasets/v3.0.2/en/package_reference/main_classes#datasets.Dataset.remove_columns) 方法移除其他列：


In [ ]:
minds = minds.remove_columns(["path", "transcription", "english_transcription", "lang_id"])


现在查看一个示例：


In [ ]:
minds["train"][0]


输出：


这里有两个字段：

- `audio`：一维 `array` 的语音信号，需要调用来加载和重采样音频文件。
- `intent_class`：表示说话人意图的类别 ID。

为了方便模型从标签 ID 获取标签名称，创建一个字典，将标签名称映射到整数，反之亦然：


In [ ]:
labels = minds["train"].features["intent_class"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label


现在你可以将标签 ID 转换为标签名称：


In [ ]:
id2label[str(2)]


输出：


## 预处理

下一步是加载一个 Wav2Vec2 特征提取器来处理音频信号：


In [ ]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")


MInDS-14 数据集的采样率为 8000Hz（你可以在其 [数据集卡片](https://huggingface.co/datasets/PolyAI/minds14) 中找到这些信息），这意味着你需要将数据集重采样到 16000Hz 以使用预训练的 Wav2Vec2 模型：


In [ ]:
minds = minds.cast_column("audio", Audio(sampling_rate=16000))
minds["train"][0]


输出：


现在创建一个预处理函数，该函数：

1. 调用 `audio` 列来加载音频文件，并在必要时重采样。
2. 检查音频文件的采样率是否与模型预训练时的采样率匹配。你可以在 Wav2Vec2 的 [模型卡片](https://huggingface.co/facebook/wav2vec2-base) 中找到这些信息。
3. 设置最大输入长度，以便在不截断的情况下批量处理较长的输入。


In [ ]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs


为了在整个数据集上应用预处理函数，使用 🤗 Datasets 的 [map](https://huggingface.co/docs/datasets/v3.0.2/en/package_reference/main_classes#datasets.Dataset.map) 函数。通过设置 `batched=True` 可以加快 `map` 的速度，以便一次处理多个数据集元素。移除不需要的列，并将 `intent_class` 重命名为 `label`，因为这是模型期望的名称：


In [ ]:
encoded_minds = minds.map(preprocess_function, remove_columns="audio", batched=True)
encoded_minds = encoded_minds.rename_column("intent_class", "label")


## 评估

在训练过程中包含一个评估指标通常有助于评估模型的性能。你可以使用 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) 库快速加载评估方法。对于此任务，加载 [准确率](https://huggingface.co/spaces/evaluate-metric/accuracy) 指标（有关如何加载和计算指标的更多信息，请参阅 🤗 Evaluate 的 [快速入门](https://huggingface.co/docs/evaluate/a_quick_tour)）：


In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")


然后创建一个函数，将预测值和标签传递给 `compute` 以计算准确率：


In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)


你现在已经准备好了 `compute_metrics` 函数，稍后在设置训练时会用到它。

## 训练

如果你不熟悉使用 [Trainer](/docs/transformers/v4.46.0/en/main_classes/trainer#transformers.Trainer) 微调模型，请参考 [这里](../training#train-with-pytorch-trainer) 的基本教程！

你现在可以开始训练模型了！使用 [AutoModelForAudioClassification](/docs/transformers/v4.46.0/en/model_doc/auto#transformers.AutoModelForAudioClassification) 加载 Wav2Vec2，并指定预期的标签数量和标签映射：


In [ ]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)


此时只剩下三个步骤：

1. 在 [TrainingArguments](/docs/transformers/v4.46.0/en/main_classes/trainer#transformers.TrainingArguments) 中定义训练超参数。唯一必需的参数是 `output_dir`，它指定了保存模型的位置。通过设置 `push_to_hub=True` 可以将模型推送到 Hub（需要登录 Hugging Face 才能上传模型）。每个 epoch 结束时，[Trainer](/docs/transformers/v4.46.0/en/main_classes/trainer#transformers.Trainer) 将评估准确率并保存训练检查点。
2. 将训练参数传递给 [Trainer](/docs/transformers/v4.46.0/en/main_classes/trainer#transformers.Trainer)，并传入模型、数据集、分词器、数据收集器和 `compute_metrics` 函数。
3. 调用 [train()](/docs/transformers/v4.46.0/en/main_classes/trainer#transformers.Trainer.train) 来微调模型。


In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_mind_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_minds["train"],
    eval_dataset=encoded_minds["test"],
    processing_class=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()


训练完成后，使用 [push_to_hub()](/docs/transformers/v4.46.0/en/main_classes/trainer#transformers.Trainer.push_to_hub) 方法将模型推送到 Hub，以便每个人都可以使用你的模型：


In [ ]:
trainer.push_to_hub()


有关如何微调模型进行音频分类的更详细示例，请参阅相应的 [PyTorch 笔记本](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/audio_classification.ipynb)。

## 推理

很好，现在你已经微调了一个模型，可以使用它进行推理了！

加载一个你想进行推理的音频文件。如果需要，记得将音频文件的采样率调整为模型的采样率！


In [ ]:
from datasets import load_dataset, Audio

dataset = load_dataset("PolyAI/minds14", name="en-US", split="train")
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
sampling_rate = dataset.features["audio"].sampling_rate
audio_file = dataset[0]["audio"]["path"]


最简单的方法是使用 [pipeline()](/docs/transformers/v4.46.0/en/main_classes/pipelines#transformers.pipeline) 进行推理。实例化一个用于音频分类的 `pipeline`，并将你的音频文件传递给它：


In [ ]:
from transformers import pipeline

classifier = pipeline("audio-classification", model="stevhliu/my_awesome_minds_model")
classifier(audio_file)


输出：


你也可以手动复制 `pipeline` 的结果：

加载一个特征提取器来预处理音频文件，并返回 `input` 作为 PyTorch 张量：


In [ ]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("stevhliu/my_awesome_minds_model")
inputs = feature_extractor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")


将输入传递给模型并返回 logits：


In [ ]:
from transformers import AutoModelForAudioClassification

model = AutoModelForAudioClassification.from_pretrained("stevhliu/my_awesome_minds_model")
with torch.no_grad():
    logits = model(**inputs).logits


获取概率最高的类别，并使用模型的 `id2label` 映射将其转换为标签：


In [ ]:
import torch

predicted_class_ids = torch.argmax(logits).item()
predicted_label = model.config.id2label[predicted_class_ids]
predicted_label


输出：